# Import necessary packages and load data


In [ ]:
import pandas as pd

df = pd.read_csv('ls_new.csv')

df

# Exploratory Data Analysis


In [ ]:
df['ReceiptNumber'].nunique()

In [ ]:
items_to_remove = ['Paper Checkout Bags','Discount', 'Clearance']

filtered_df = df[~df['Details'].isin(items_to_remove)]

## Popularity by Category

In [ ]:
# Finding the most popular in chocolate buttons
# Filter the DataFrame for rows where 'Details' includes 'Chocolate Buttons'
chocolate_buttons_df = filtered_df[filtered_df['Details'].str.contains('Chocolate Buttons', case=False, na=False)]

# Calculate the sum of the 'Quantity' column for these rows
total_quantity = chocolate_buttons_df['Quantity'].sum()



In [ ]:
chocolate_buttons_df['Details'].value_counts()

In [ ]:
# Most popular jelly beans
# # Filter the DataFrame for rows where 'Details' includes 'Chocolate Buttons'
jelly_beans_df = filtered_df[filtered_df['Details'].str.contains('Jelly Beans', case=False, na=False)]

In [ ]:
jelly_beans_df['Details'].value_counts()

In [ ]:
# Filter the DataFrame for rows where 'Details' includes 'lollipops'
lollipops_df = filtered_df[filtered_df['Details'].str.contains('Lollipops', case=False, na=False)]

In [ ]:
lollipops_df['Details'].value_counts()

In [ ]:
#Licorice allsorts
lico_allsorts = filtered_df[filtered_df['Details'].str.contains('Licorice Allsorts', case=False, na=False)]

In [ ]:
lico_allsorts['Details'].value_counts()

In [ ]:
# Warheads soda popularity and units sold
warheads = filtered_df[filtered_df['Details'].str.contains('Warheads Sour Soda Can  355ml', case=False, na=False)]

In [ ]:
warheads['Details'].value_counts()

In [ ]:
# Most popular in Dutch Licorice
dc = filtered_df[filtered_df['Details'].str.contains('Dutch Licorice', case=False, na=False)]

In [ ]:
dc['Details'].value_counts()

In [ ]:
# Ensure 'Quantity' is integer and handle NaNs
filtered_df['Quantity'] = filtered_df['Quantity'].fillna(0).astype(int)

# Repeat 'Details' by 'Quantity' to calculate the real item quantity sold in each row
repeated_details = filtered_df.apply(lambda row: [row['Details']] * row['Quantity'], axis=1)

# Flatten the list of lists
flattened_details = [item for sublist in repeated_details for item in sublist]

# Calculate value counts
details_counts = pd.Series(flattened_details).value_counts().reset_index()

details_counts.columns = ['NameofProduct', 'PurchaseFrequencyOverall']

In [ ]:
# Creating an explicit copy if df is a slice of another DataFrame
df_copy = details_counts.copy()

# Function to clean the product names
def clean_product_name(name):
    parts = name.split(' / ')
    return parts[0] if len(parts) > 1 else name

# Apply the function to the DataFrame using .loc
df_copy.loc[:, 'NameofProduct'] = df_copy['NameofProduct'].apply(clean_product_name)

# Display the updated DataFrame
print(df_copy)

In [ ]:
df_copy.rename(columns={'NameofProduct': 'name'}, inplace=True)

# Exporting Product Category

In [ ]:
categories = pd.read_csv('product-export.csv')
categories

In [ ]:
categories_small = categories[['name','product_category']]

In [ ]:
categories_no_duplicates = categories_small.drop_duplicates(subset=['name'])

# Merging Transactions with Product Categories

In [ ]:
merged_df = pd.merge(df_copy, categories_no_duplicates, how='left', on='name')

In [ ]:
merged_df

In [ ]:
merged_df.to_csv('popularcategories.csv')

In [ ]:
# Convert 'product_category' NaN values to a placeholder if necessary (e.g., 'Unknown')
merged_df['product_category'].fillna('Unknown', inplace=True)

# Group by 'product_category' and sum the 'PurchaseFrequencyOverall'
category_summary = merged_df.groupby('product_category')['PurchaseFrequencyOverall'].sum().reset_index()

# Rename columns for clarity
category_summary.columns = ['ProductCategory', 'TotalProductsSold']

# Print the summary DataFrame
print(category_summary)

In [ ]:
category_summary.to_clipboard(index=False)